## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kapaa,US,22.0752,-159.3190,84.52,66,40,13.80,scattered clouds
1,1,Rikitea,PF,-23.1203,-134.9692,70.52,69,58,20.78,broken clouds
2,2,Harper,LR,4.3750,-7.7169,75.58,87,42,7.74,scattered clouds
3,3,Linqing,CN,36.8456,115.7117,67.73,65,96,6.46,overcast clouds
4,4,Tuatapere,NZ,-46.1333,167.6833,44.53,88,99,12.68,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kapaa,US,22.0752,-159.3190,84.52,66,40,13.80,scattered clouds
2,2,Harper,LR,4.3750,-7.7169,75.58,87,42,7.74,scattered clouds
5,5,Itarema,BR,-2.9248,-39.9167,78.03,81,44,18.37,scattered clouds
12,12,Nhulunbuy,AU,-12.2333,136.7667,75.81,78,75,18.41,broken clouds
14,14,Ixtapa,MX,20.7000,-105.2000,87.03,70,75,6.91,broken clouds
24,24,Mayo,US,38.8876,-76.5119,80.94,71,0,16.11,clear sky
34,34,Atuona,PF,-9.8000,-139.0333,77.07,74,28,23.38,scattered clouds
35,35,Singaraja,ID,-8.1120,115.0882,80.65,71,14,3.22,few clouds
37,37,Vaini,TO,-21.2000,-175.2000,75.36,78,20,13.80,few clouds
40,40,Naze,JP,28.3667,129.4833,80.06,81,98,22.06,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        210
City           210
Country        210
Lat            210
Lng            210
Max Temp       210
Humidity       210
Cloudiness     210
Wind Speed     210
Description    210
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
2,Harper,LR,75.58,scattered clouds,4.3750,-7.7169,
5,Itarema,BR,78.03,scattered clouds,-2.9248,-39.9167,
12,Nhulunbuy,AU,75.81,broken clouds,-12.2333,136.7667,
14,Ixtapa,MX,87.03,broken clouds,20.7000,-105.2000,
24,Mayo,US,80.94,clear sky,38.8876,-76.5119,
34,Atuona,PF,77.07,scattered clouds,-9.8000,-139.0333,
35,Singaraja,ID,80.65,few clouds,-8.1120,115.0882,
37,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
40,Naze,JP,80.06,light rain,28.3667,129.4833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))